# EIRA: Hazard class

### @Authors, last update july 2024
- Jose Antonio Leon
- Luc Marius Jacques Bonnafous
- Natalia Romero


#### What is a hazard?
A hazard describes disaster or catastrophic events such as earthquakes, storms, floods, droughts, heat waves.

The hazard in EIRA can be described using diffent options:
- in terms of the return period and a physical intensity field 
- in terms of the frequency of ocurrence and physical instensity field
- in terms of the probability of occurrence and physical instensity field

<br>

#### What do hazards look like in EIRA?

Decripcion of hazards `Hazard` 

| Mandatory variables &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Data Type &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Description &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
| :- | :- | :- |
| units |(str)| units of the intensity|
| centroids |`Centroids()`| centroids of the events|
| event_id |(np.array)| id (>0) of each event|
| frequency |(np.array)| frequency of each event in years|
| intensity |(sparse.csr_matrix)| intensity of the events at centroids|
| fraction |(sparse.csr_matrix)| fraction of affected exposures for each event at each centroid|

<br>
<br>

| Descriptive variables | Data Type | Description|
| :- | :- | :- |
| date |(np.array)| integer date corresponding to the proleptic Gregorian ordinal, where January 1 of year 1 has<br> ordinal 1 (ordinal format of datetime library)|
| orig |(np.array)| flags indicating historical events (True) or probabilistic (False) |
| event_name |(list(str))| name of each event (default: event_id)|
<br>

Note that `intensity` and `fraction` are scipy.sparse matrices of size num_events x num_centroids. The `fraction` attribute is optional. The `Centroids` class contains the geographical coordinates where the hazard is defined. A `Centroids` instance provides the coordinates either as points or raster data together with their Coordinate Reference System (CRS). The default CRS used in climada is the usual EPSG:4326. `Centroids` provides moreover methods to compute centroids areas, on land mask, country iso mask or distance to coast

## EIRA Features

## EIRA Classes

## Tutorial: an example of risk assessment using EIRA

This example will work through a risk assessment for floods in San Salvador, constructing hazard, exposure and vulnerability and ovelaping them to compute certain indicator of risk. Everything you need for this is included in the main EIRA installation and additional data will be downloaded by the scripts as required.

## Hazards

Hazards are characterized by their frequency of occurrence and the geographical distribution of their intensity. The `Hazard` class collects events of the same hazard type (e.g. tropical cyclone, flood, drought, ...) with intensity values over the same geographic centroids. They might be historical events or synthetic.

### Import libraries

In [1]:
# import libraries
import json                        

import os                                    #this code is very important  
import sys                                   #this line is very important  
sys.path.append(os.path.join( '..'))         #this line is very important to call the other modules as a package and use the import option. You need the other two line above.

from eira import EIRA_GIS_tools
from eira import EIRA_NotebookUserInter
from eira import EIRA_files_handler
import rasterio

import ipywidgets as widgets
from IPython.display import display
import pandas as pd


### Select the country of analysis

In [ ]:
root_folder_path = os.path.abspath('Auxname.txt') 
print(root_folder_path)
country_list_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/List_of_Countries_database.csv'

dropdown = EIRA_NotebookUserInter.create_dropdown(country_list_path,"NAME_LONG", ';')
EIRA_NotebookUserInter.check_if_selection_changed(dropdown, 'Country selected: ')

### Files to read

In [ ]:
# Get the path of the root folder
root_folder_path = os.path.abspath('Auxname.txt')   #this is a code to get the direction of the root folder: Auxname is jus an auxiliar

# Raster file path
#raster_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/pc_h2glob_ELS.tif'
#raster_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/global_pc_h2globGuatemalaElSalvador.tif'
raster_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/global_pc_h2globCentralAmerica.tif'

grd_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/Earthquakes/Seismic hazard_PGA_RT475years.grd'

outputdir = '/root/WB/EIRA_tool/EIRA/eira/output_data/test1.tif'

# Shape file path
shape_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/Country_adm_boundaries/gadm36_SLV_shp/gadm36_SLV_0.shp'
#shape_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/Country_adm_boundaries/gadm41_SLV.gpkg'
#shape_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/Country_adm_boundaries/world-administrative-boundariesOpendatasoft/world-administrative-boundaries.shp'
#shape_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/Country_adm_boundaries/MapWord/MapWord_admin_0.shp'

ra = EIRA_GIS_tools.read_raster_file(raster_file_path)
EIRA_GIS_tools.plot_raster_in_memory(ra)

ra2 = EIRA_GIS_tools.read_gdr_file(grd_file_path)
EIRA_GIS_tools.plot_raster_in_memory(ra2)

gdf = EIRA_GIS_tools.read_and_reproject_vector_file(shape_file_path)
EIRA_GIS_tools.plot_vectorGIS_in_memory(gdf)

#aster = EIRA_GIS_tools.read_and_reproject_raster_file(grd_file_path)
raster = EIRA_GIS_tools.read_and_reproject_grd_file(grd_file_path)
EIRA_GIS_tools.plot_raster_in_memory(raster)

#raster2, metadaraster2 = EIRA_GIS_tools.extract_raster_by_vectorGISfile_mask(ra2,gdf,outputdir)   
raster3= EIRA_GIS_tools.extract_raster_by_vectorGISfile_mask(ra2,gdf)
EIRA_GIS_tools.plot_raster_in_memory(raster3)

#EIRA_GIS_tools.plot_raster_and_shapefile(raster,metadaraster2,gdf)
EIRA_GIS_tools.plot_raster_and_shapefile(raster3,gdf)

In [ ]:
# Shape file path
shape_file_path1 = '/root/WB/EIRA_tool/EIRA/eira/input_data/Exposure/ExpoPuentesELS.shp'
shape_file_path2 = '/root/WB/EIRA_tool/EIRA/eira/input_data/Exposure/ExtractionlayerELS.shp'

gdf1 = EIRA_GIS_tools.read_vectorGIS_file(shape_file_path1)
gdf2=EIRA_GIS_tools.read_vectorGIS_file(shape_file_path2)

EIRA_GIS_tools.plot_vectorGIS_in_memory(gdf1)
EIRA_GIS_tools.plot_vectorGIS_in_memory(gdf2)

extracteshape = EIRA_GIS_tools.extract_vectorGIS_by_vectorGISfile_mask(gdf1,gdf2)

listofgdf = [gdf2,extracteshape]
EIRA_GIS_tools.plot_geodataframes(listofgdf)

#shape_file_path = '/root/WB/EIRA_tool/EIRA/eira/input_data/Country_adm_boundaries/gadm41_SLV.gpkg'

In [2]:
# Download a file in internet
IBTRACS_URL = ('https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/netcdf/')
IBTRACS_FILE = 'IBTrACS.ALL.v04r00.nc'
urlJALT = (f'{IBTRACS_URL}/{IBTRACS_FILE}')
print (urlJALT)
folderpath = '/root/WB/EIRA_tool/EIRA/eira/output_data'
urlJALT= 'https://onedrive.live.com/download?resid=23323221D505E66D%2183100&authkey=%21AP1zlT6wiXzI8C0&'
#EIRA_files_handler.download_file(urlJALT,forderpath)
urlJALT = ' https://onedrive.live.com/download?resid=23323221D505E66D%2183100&authkey=%21AP1zlT6wiXzI8C0&'
EIRA_files_handler.download_onedrive_file(urlJALT,'pc_h2glob_ELS.tif',folderpath)

#EIRA_files_handler.download_onedrive_file(urlJALT,forderpath)
urlJALT ='https://onedrive.live.com/download?resid=23323221D505E66D!84295&authkey=!AKUqSpH2oIhnt_E'
EIRA_files_handler.download_onedrive_file(urlJALT,'Seismic hazard_PGA_RT475years.grd',forderpath)


https://www.ncei.noaa.gov/data/international-best-track-archive-for-climate-stewardship-ibtracs/v04r00/access/netcdf//IBTrACS.ALL.v04r00.nc


'/root/WB/EIRA_tool/EIRA/eira/output_data/pc_h2glob_ELS.tif'